# Gaussian Distribution
This notebook contains functions that plot 1D and 2D Gaussian distributions. The goal is to understand how a probability density function (PDF) is created, and to visualize Gaussians and see how they change in space given different parameters.

[Check my Github repository for similar introductory notebooks](https://github.com/YZouzou/ML-Topics-Intro)

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import interact
import ipywidgets as widgets
plt.style.use('bmh')
# %matplotlib notebook

### Function
### `Gaussian_1d`:
This function takes in a value `x`, a mean `mu`, and a standard deviation `sig`. The function returns the value corresponding to `x` in a univariate Gaussian PDF with a mean equal to `mu` and a standard deviation equal to `sig`.

In [2]:
def Gaussian_1d(x, mu, sig):
    return (2 * np.pi * sig**2)**(-0.5) * np.exp(-0.5 * ((x - mu) / sig)**2)

In [20]:
@interact(mu = widgets.IntSlider(min=-3, max=3, step=1, value=0, description = '\u03BC (Mean)'),
         sig = widgets.FloatSlider(min=0.3, max=1.2, step=0.1, value=0.7, description = '\u03C3 (Std)'))
def plot_1D_gaussian(mu, sig):
    x = np.linspace(-5, 5, 1000)
    y = Gaussian_1d(x, mu, sig)

    plt.figure(figsize = (8, 6))
    plt.plot(x, y)
    plt.xlim(-5, 5)
    plt.ylim(0, 1.4)
    plt.title('Univariate Gaussian Distribution')
    plt.show()

interactive(children=(IntSlider(value=0, description='μ (Mean)', max=3, min=-3), FloatSlider(value=0.7, descri…

### Function
### `Gaussian_2d`:
This function takes in a 2x1 vector `X`, a 2x1 mean vector `Mu`, and a 2x2 covariance matrix `Sig`. The function returns the value corresponding to vector `X` in a bivariate Gaussian PDF with a mean vector equal to `Mu` and a covariance matrix equal to `Sig`.

In [30]:
def Gaussian_2d(X, Mu, Sig):
    return np.linalg.det(2 * np.pi * Sig)**(-0.5) * np.exp(-0.5 * (X - Mu).T @ np.linalg.inv(Sig) @ (X - Mu))

In [33]:
# Number of points (Smoothness of graph)
n = 100

X = np.linspace(-5, 5, n)
Y = np.linspace(-5, 5, n)

X, Y = np.meshgrid(X, Y)

# Mean vector
Mu = np.array([0, 0]).reshape(-1, 1)

# Covariance matrix
Sig = np.array([
    [3, 2],
    [2, 3]])

Timing different methods for calculating Z matrix to create the plot

In [52]:
%%time
# Creating an array whereby the first row contains X values and the second row Y values
# Z values are calculated by iterating through every single (x, y) vector

inp = np.array([X.flatten(), Y.flatten()])

Z1 = np.zeros(X.shape).flatten()
for i, x in enumerate(inp.T):
    Z1[i] = Gaussian_2d(x.reshape(-1, 1), Mu, Sig)

Z1 = Z1.reshape(X.shape)
display(Z1)

array([[4.79581829e-04, 5.28934034e-04, 5.79804564e-04, ...,
        2.68124826e-12, 1.63299326e-12, 9.88491825e-13],
       [5.28934034e-04, 5.85750616e-04, 6.44711358e-04, ...,
        4.39342724e-12, 2.68672522e-12, 1.63299326e-12],
       [5.79804564e-04, 6.44711358e-04, 7.12508974e-04, ...,
        7.15502532e-12, 4.39342724e-12, 2.68124826e-12],
       ...,
       [2.68124826e-12, 4.39342724e-12, 7.15502532e-12, ...,
        7.12508974e-04, 6.44711358e-04, 5.79804564e-04],
       [1.63299326e-12, 2.68672522e-12, 4.39342724e-12, ...,
        6.44711358e-04, 5.85750616e-04, 5.28934034e-04],
       [9.88491825e-13, 1.63299326e-12, 2.68124826e-12, ...,
        5.79804564e-04, 5.28934034e-04, 4.79581829e-04]])

Wall time: 305 ms


In [53]:
%%time
# Create an array of n rows, in each row there is a matrix of 2 rows and n columns
# The Gaussian function will iterate through every matrix of these

mat = np.zeros((n, 2, n))

# Assign X values to the first row of every matrix
mat[:, 0, :] = X
# Assign Y values to the second row of every matrix
mat[:, 1, :] = Y

Z2 = np.zeros((n, n))

for i in range(n):
    # The values of the bivariate gaussian distribution are in the diagonal of the resulting matrix
    Z2[i, :] = np.diag(Gaussian_2d(mat[i, :, :], Mu, Sig))
display(Z2)

array([[4.79581829e-04, 5.28934034e-04, 5.79804564e-04, ...,
        2.68124826e-12, 1.63299326e-12, 9.88491825e-13],
       [5.28934034e-04, 5.85750616e-04, 6.44711358e-04, ...,
        4.39342724e-12, 2.68672522e-12, 1.63299326e-12],
       [5.79804564e-04, 6.44711358e-04, 7.12508974e-04, ...,
        7.15502532e-12, 4.39342724e-12, 2.68124826e-12],
       ...,
       [2.68124826e-12, 4.39342724e-12, 7.15502532e-12, ...,
        7.12508974e-04, 6.44711358e-04, 5.79804564e-04],
       [1.63299326e-12, 2.68672522e-12, 4.39342724e-12, ...,
        6.44711358e-04, 5.85750616e-04, 5.28934034e-04],
       [9.88491825e-13, 1.63299326e-12, 2.68124826e-12, ...,
        5.79804564e-04, 5.28934034e-04, 4.79581829e-04]])

Wall time: 14 ms


In [54]:
%%time
# Using Scipy function
from scipy.stats import multivariate_normal

pos = np.zeros((X.shape[0], X.shape[0], 2))
pos[:,:,0]=X
pos[:,:,1]=Y

rv = multivariate_normal(Mu.flatten(), Sig)

Z_scipy = rv.pdf(pos)
display(Z_scipy)

array([[4.79581829e-04, 5.28934034e-04, 5.79804564e-04, ...,
        2.68124826e-12, 1.63299326e-12, 9.88491825e-13],
       [5.28934034e-04, 5.85750616e-04, 6.44711358e-04, ...,
        4.39342724e-12, 2.68672522e-12, 1.63299326e-12],
       [5.79804564e-04, 6.44711358e-04, 7.12508974e-04, ...,
        7.15502532e-12, 4.39342724e-12, 2.68124826e-12],
       ...,
       [2.68124826e-12, 4.39342724e-12, 7.15502532e-12, ...,
        7.12508974e-04, 6.44711358e-04, 5.79804564e-04],
       [1.63299326e-12, 2.68672522e-12, 4.39342724e-12, ...,
        6.44711358e-04, 5.85750616e-04, 5.28934034e-04],
       [9.88491825e-13, 1.63299326e-12, 2.68124826e-12, ...,
        5.79804564e-04, 5.28934034e-04, 4.79581829e-04]])

Wall time: 4.99 ms


Checking that Z2 is equal to Z_scipy

In [57]:
(np.abs(Z2 - Z_scipy) > 1e-16).sum()

0

### Visualizing bivariate Gaussians
Note that an error may occur if the values result in a non-positive semi-definite covariance matrix.

In [125]:
@interact(\
          mu1 = (-2, 2, 1),
          mu2 = (-2, 2, 1),
          sig11 = widgets.FloatSlider(min = 0.4, max = 2.4, step = 0.2, value = 1),
          sig22 = widgets.FloatSlider(min = 0.4, max = 2.4, step = 0.2, value = 1),
          sig12 = widgets.FloatSlider(min = -1.2, max = 1.2, step = 0.2, value = 0)
)
def plot_2D_gaussian(mu1, mu2, sig11, sig22, sig12):
    Mu = np.array([mu1, mu2])
    Sig = np.array([
        [sig11, sig12],
        [sig12, sig22]
    ])
    
    rv = multivariate_normal(Mu, Sig)
    Z_scipy = rv.pdf(pos)

    fig = plt.figure(figsize = (10, 10))

    ax1 = fig.add_subplot(2, 2, 1, projection = '3d')
    ax2 = fig.add_subplot(2, 2, 2, projection = '3d')
    ax3 = fig.add_subplot(2, 2, 3, projection = '3d')
    ax4 = fig.add_subplot(2, 2, 4, projection = '3d')
    axes = [ax1, ax2, ax3, ax4]

    azims = [45, 270, 0]
    elevs = [20, 0, 0]
    titles = []

    for i, ax in enumerate(axes[:3]):
        ax.plot_surface(X, Y, Z_scipy, cmap="gray")
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.azim = azims[i]
        ax.elev = elevs[i]
        ax.set_zlim(0, 0.3)
    
    ax4.contour(X, Y, Z_scipy, cmap="gray")
    ax4.set_xlabel('X')
    ax4.set_ylabel('Y')
    ax4.azim = 270
    ax4.elev = 90
    ax4.dist = 7


    plt.show()

interactive(children=(IntSlider(value=0, description='mu1', max=2, min=-2), IntSlider(value=0, description='mu…